In [ ]:
# Task-05: Network Packet Analyzer (Colab-friendly)
# - Option A: Upload & analyze a .pcap file (recommended for Colab)
# - Option B: Generate simulated packets and analyze (for demo)
# - Option C: Live sniffing example for local machine (NOT for Colab)

# Install scapy (used for pcap parsing). This works in Colab.
!pip install -q scapy pandas

from scapy.all import rdpcap, IP, TCP, UDP, Raw, Ether
import pandas as pd
from google.colab import files
import io
import time

def analyze_packets(pkts):
    """
    Parse a list of scapy packets and return a DataFrame summarizing:
    timestamp, src, dst, protocol, sport, dport, payload_preview, length
    """
    rows = []
    for pkt in pkts:
        ts = getattr(pkt, 'time', None)
        length = len(pkt) if hasattr(pkt, '__len__') else None
        src = dst = proto = sport = dport = payload = None

        # Try Ethernet -> IP
        if IP in pkt:
            ip = pkt[IP]
            src = ip.src
            dst = ip.dst

            if TCP in pkt:
                proto = 'TCP'
                sport = pkt[TCP].sport
                dport = pkt[TCP].dport
                if Raw in pkt:
                    payload = bytes(pkt[Raw].load)
            elif UDP in pkt:
                proto = 'UDP'
                sport = pkt[UDP].sport
                dport = pkt[UDP].dport
                if Raw in pkt:
                    payload = bytes(pkt[Raw].load)
            else:
                proto = ip.proto  # numeric fallback
        else:
            # Non-IP packets (ARP, etc.)
            if pkt.haslayer(Ether):
                src = pkt[Ether].src
                dst = pkt[Ether].dst
                proto = pkt[Ether].type

        # Make a small, safe preview of payload (printable ASCII up to 50 bytes)
        payload_preview = None
        if payload:
            try:
                # show only printable area and limit length
                preview = payload[:200]
                # decode safely
                payload_preview = preview.decode('utf-8', errors='replace')
                # shorten long previews
                if len(payload_preview) > 100:
                    payload_preview = payload_preview[:100] + '...'
            except Exception:
                payload_preview = repr(payload[:50])

        rows.append({
            "timestamp": time.ctime(ts) if ts else None,
            "src": src,
            "dst": dst,
            "protocol": proto,
            "sport": sport,
            "dport": dport,
            "length": length,
            "payload_preview": payload_preview
        })

    df = pd.DataFrame(rows)
    return df

# -------------------------
# Option A: Upload & analyze a pcap
# -------------------------
def option_analyze_pcap():
    print("🔽 Upload a .pcap file (from Wireshark / tcpdump).")
    print("After uploading, the notebook will parse and summarize packets.")
    uploaded = files.upload()
    if not uploaded:
        print("No file uploaded.")
        return

    # pick the first uploaded file
    filename = next(iter(uploaded))
    print(f"Uploaded: {filename}  ({len(uploaded[filename])} bytes)")

    # Read pcap into scapy
    try:
        pkts = rdpcap(io.BytesIO(uploaded[filename]))
    except Exception as e:
        print("Error reading pcap:", e)
        return

    print(f"Total packets read: {len(pkts)}")
    df = analyze_packets(pkts)

    # show top rows and protocol counts
    display(df.head(30))
    print("\nProtocol counts:")
    print(df['protocol'].value_counts(dropna=False))

    # Save summary to CSV so you can download
    csv_name = "pcap_summary.csv"
    df.to_csv(csv_name, index=False)
    print(f"\nSaved summary to {csv_name}. Use the link below to download.")
    files.download(csv_name)


# -------------------------
# Option B: Simulate packet list & analyze (demo in Colab)
# -------------------------
def option_simulate_and_analyze():
    from scapy.layers.inet import IP, TCP, UDP
    from scapy.packet import Packet

    print("Creating a small simulated packet list for demo...")

    # Build simple packets (note: these are scapy Packet objects, not sent on the wire)
    pkt1 = IP(src="192.168.1.10", dst="93.184.216.34")/TCP(sport=44444, dport=80)/Raw(load=b"GET / HTTP/1.1\r\nHost: example.com\r\n\r\n")
    pkt2 = IP(src="10.0.0.5", dst="8.8.8.8")/UDP(sport=55555, dport=53)/Raw(load=b"\x12\x34\x01\x00")
    pkt3 = IP(src="172.16.0.2", dst="172.16.0.1")/TCP(sport=22, dport=53421)/Raw(load=b"ssh-data-sample")
    pkts = [pkt1, pkt2, pkt3]

    df = analyze_packets(pkts)
    display(df)
    print("\nProtocol counts:")
    print(df['protocol'].value_counts(dropna=False))

# -------------------------
# Option C: Live sniffing example (local only)
# -------------------------
live_sniffing_snippet = r'''
# === Live sniffing (RUN LOCALLY, requires root/admin privileges) ===
# Save as a .py and run with sudo if needed:
# sudo python3 live_sniff.py

from scapy.all import sniff, IP, TCP, UDP, Raw
import time

def handle_packet(pkt):
    ts = time.ctime(pkt.time)
    src = pkt[IP].src if IP in pkt else None
    dst = pkt[IP].dst if IP in pkt else None
    proto = None
    if TCP in pkt:
        proto = 'TCP'
    elif UDP in pkt:
        proto = 'UDP'
    else:
        proto = pkt.summary()
    payload_preview = pkt[Raw].load[:100] if Raw in pkt else b''
    print(f"{ts} {src} -> {dst} {proto} payload_preview={payload_preview}")

# sniff 100 packets on interface 'eth0' (change interface as needed)
sniff(count=100, prn=handle_packet, store=0)
'''

# -------------------------
# User menu in Colab
# -------------------------
print("Task-05: Network Packet Analyzer (Colab-friendly)\n")
print("Choose an option:")
print("1) Upload & analyze a pcap file (recommended for Colab)")
print("2) Run a simulated packet demo (no upload)")
print("3) Show live-sniffing code to run on your local machine (NOT for Colab)")
choice = input("Enter 1, 2, or 3: ").strip()

if choice == "1":
    option_analyze_pcap()
elif choice == "2":
    option_simulate_and_analyze()
elif choice == "3":
    print("\n=== Live sniffing snippet (run locally with sudo) ===\n")
    print(live_sniffing_snippet)
else:
    print("Invalid choice. Please run the cell again and choose 1, 2, or 3.")


Task-05: Network Packet Analyzer (Colab-friendly)

Choose an option:
1) Upload & analyze a pcap file (recommended for Colab)
2) Run a simulated packet demo (no upload)
3) Show live-sniffing code to run on your local machine (NOT for Colab)
Enter 1, 2, or 3: 2
Creating a small simulated packet list for demo...


,timestamp,src,dst,protocol,sport,dport,length,payload_preview
0,Fri Aug 29 13:47:36 2025,192.168.1.10,93.184.216.34,TCP,44444,80,77,GET / HTTP/1.1\r\nHost: example.com\r\n\r\n
1,Fri Aug 29 13:47:36 2025,10.0.0.5,8.8.8.8,UDP,55555,53,32,4 
2,Fri Aug 29 13:47:36 2025,172.16.0.2,172.16.0.1,TCP,22,53421,55,ssh-data-sample



Protocol counts:
protocol
TCP    2
UDP    1
Name: count, dtype: int64
